In [ ]:
import psycopg2
from passwords import user, passw # include a file to configure these for your own use
from datetime import date, datetime
import json

In [49]:
def makeJSON(records): # Custom 'jsonify' function, takes list of lists, makes list of dictionaries
    output = []
    for i in range(len(records)):
        temp_out = {
            'week' : records[i][0],
            'state' : records[i][1],
            'week_start' : records[i][2],
            'covid_d' : records[i][3],
            'pneum_d' : records[i][4],
            'flu_d' : records[i][5],
            'anyof3_d' : records[i][6],
            'total_d' : records[i][7],
            'poe_d' : records[i][8],
            'year': records[i][9]}
        output.append(temp_out)
    return output

In [50]:
def pickState(state): # Picks all data for chosen state
    sql = "select * from covid where state like (%s)"
    data = (state,)
    cur.execute(sql, data)
    records = cur.fetchall()
    output = makeJSON(records)
    return output


In [51]:
def pickYear(year): # Picks all data for chosen year
    sql = "select * from covid where year = (%s)"
    data = (year,)
    cur.execute(sql, data)
    records = cur.fetchall()
    output = makeJSON(records)
    return output

In [52]:
def makeJSON2(records): # Same thing as the first but for the other DB
    output = []
    for i in range(len(records)):
        temp_out = {
            'indicator':records[i][0],
            'group':records[i][1],
            'state':records[i][2],
            'subgroup':records[i][3],
            'time_start':records[i][4],
            'time_end':records[i][5],
            'value':records[i][6],
            'low':records[i][7],
            'high':records[i][8]
        }
        output.append(temp_out)
    return output

In [53]:
def getYearM(year): # Same again for other DB
    sql = "select * from mental where time_start between (%s) and (%s)"
    data = (f"{year-1}-01-01",f"{year+1}-01-01",)
    cursor2.execute(sql, data)
    records = cursor2.fetchall()
    output = makeJSON2(records)
    return output


In [54]:
def getYearM(group): # Grab data for a specific group in the 'divisor' column
    sql = "select * from mental where divisor like (%s)"
    data = (group.title(),)
    cursor2.execute(sql, data)
    records = cursor2.fetchall()
    print(records)
    output = makeJSON2(records)
    return output

In [ ]:
# Psycopg2 Setup
conn = psycopg2.connect(f"dbname=covid user={user} password={passw}")
cur = conn.cursor() # Create connection, and point at it
cur.execute('select * from covid') # Execute our sql code
records = cur.fetchall() # grab all of it, this returns a list of lists
record = makeJSON(records) # jsonify
for i in range(len(record)): # make the week var a string so we can json output
    strobj = str(record[i]['week_start'])
    record[i]['week_start'] = strobj

with open("data/json/covid_data.json", "w") as file: 
    json.dump(record, file, indent=4) # json dump

conn2 = psycopg2.connect(f"dbname=mental_health user={user} password={passw}")
cursor2 = conn2.cursor() # same stuff for other database
cursor2.execute("select * from mental")
reco = cursor2.fetchall()
jsonrec = makeJSON2(reco)
for i in range(len(jsonrec)):
    strobj = str(jsonrec[i]['time_start']) # Two date vars this time
    jsonrec[i]['time_start'] = strobj
    strobj2 = str(jsonrec[i]['time_end'])
    jsonrec[i]['time_end'] = strobj2

with open("data/json/mental_data.json", "w") as file:
    json.dump(jsonrec, file, indent=4)
